In [1]:
import sys
sys.path.insert(0,'../scripts')

import warnings

warnings.filterwarnings("ignore")
import ipywidgets
import matplotlib.pyplot as plt
import numpy as np
import torch
from PIL import Image
from torchvision.datasets import ImageFolder

import augmentations
import build_datasets
from main import parse_args

In [2]:
def to_numpy(t):
    array = torch.clip((t * 0.224) + 0.45, 0, 1).permute(1, 2, 0).numpy()
    return array

In [3]:
args = parse_args(params_path='/home/yue3/DINO/scripts/yaml/test_params.yaml')
augmentation_params = args['augmentation_params']
dataset_params = args['dataset_params']
transforms_aug = augmentations.DataAugmentationDINO(
        augmentation_params['global_crops_scale'],
        augmentation_params['local_crops_scale'],
        augmentation_params['local_crops_number'],
        augmentation_params['global_size'],
        augmentation_params['local_size']
    )




ARGS ARE: 
{'dataset_params': {'dataset_name': 'ImageNet', 'data_folder': '/storage/yue/data/'}, 'dataloader_params': {'trainloader': {'batch_size': 36, 'num_workers': 6, 'pin_memory': True, 'drop_last': True}, 'valloader': {'batch_size': 36, 'num_workers': 6, 'pin_memory': True, 'drop_last': True}}, 'augmentation_params': {'global_crops_scale': [0.4, 1.0], 'local_crops_scale': [0.05, 0.4], 'local_crops_number': 8, 'global_size': 224, 'local_size': 96}, 'model_params': {'backbone_option': 'vit_tiny', 'patch_size': 16, 'drop_path_rate': 0.1, 'out_dim': 65536, 'use_bn_in_head': False, 'norm_last_layer': False}, 'system_params': {'num_gpus': 2, 'gpu_ids': '0,1', 'random_seed': 6111}}




In [4]:
train_aug_dataset = build_datasets.get_datasets(dataset_params, 'train/', transforms_aug)

In [5]:
@ipywidgets.interact
def _(
    i=ipywidgets.IntSlider(min=0, max=len(train_aug_dataset) - 1, continuous_update=False),
    seed=ipywidgets.IntSlider(min=0, max=50, continuous_update=False),
):
    torch.manual_seed(seed)
    all_crops, _ = train_aug_dataset[i]
    titles = ["Global 1", "Global 2", "Local 1", "Local 2"]
    
    original_img = np.array(Image.open(train_aug_dataset.samples[i][0]))
    _, ax_orig = plt.subplots(figsize=(15, 5))
    ax_orig.imshow(original_img)
    ax_orig.set_title("Original")
    ax_orig.axis("off")
    
    
    fig, axs = plt.subplots(2, 2, figsize=(10, 10))
    
    for i, title in enumerate(titles):
        ax = axs[i // 2, i % 2]
        ax.imshow(to_numpy(all_crops[i]))
        ax.set_title(title)
        ax.axis("off")
    fig.tight_layout()

interactive(children=(IntSlider(value=0, continuous_update=False, description='i', max=1281166), IntSlider(val…